<a href="https://colab.research.google.com/github/andypeng93/Healthcare/blob/main/Healthcare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Flatiron School/Healthcare"
# /content/gdrive/My Drive/Kaggle is the path where kaggle.json is present in the Google Drive

In [ ]:
#changing the working directory
%cd /content/gdrive/My Drive/Flatiron School/Healthcare
#Check the present working directory using pwd command

/content/gdrive/My Drive/Flatiron School/Healthcare


In [ ]:
!kaggle datasets download -d vetrirah/av-healthcare2

 76% 5.00M/6.54M [00:00<00:00, 31.2MB/s]
100% 6.54M/6.54M [00:00<00:00, 31.1MB/s]


In [ ]:
#unzipping the zip files and deleting the zip files
!unzip \*.zip  && rm *.zip

Archive:  av-healthcare2.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               
  inflating: train_data_dict.csv     


In [ ]:
!ls

Healthcare.ipynb  sample_submission.csv  train.csv
kaggle.json	  test.csv		 train_data_dict.csv


# Imports and Loading Datasets

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import keras
import tensorflow as tf
from keras import models, layers, optimizers, regularizers
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from keras.preprocessing import image
import seaborn as sns
from sklearn.metrics import roc_curve, auc, precision_score, recall_score, accuracy_score, f1_score, confusion_matrix, classification_report, roc_auc_score
np.random.seed(123)

In [ ]:
datainfo = pd.read_csv('train_data_dict.csv')
datainfo

,Column,Description
0,case_id,Case_ID registered in Hospital
1,Hospital_code,Unique code for the Hospital
2,Hospital_type_code,Unique code for the type of Hospital
3,City_Code_Hospital,City Code of the Hospital
4,Hospital_region_code,Region Code of the Hospital
5,Available Extra Rooms in Hospital,Number of Extra rooms available in the Hospital
6,Department,Department overlooking the case
7,Ward_Type,Code for the Ward type
8,Ward_Facility_Code,Code for the Ward Facility
9,Bed Grade,Condition of Bed in the Ward


In [ ]:
dftrain = pd.read_csv('train.csv')
print(dftrain.head())
print(len(dftrain))

   case_id  Hospital_code Hospital_type_code  ...    Age Admission_Deposit   Stay
0        1              8                  c  ...  51-60            4911.0   0-10
1        2              2                  c  ...  51-60            5954.0  41-50
2        3             10                  e  ...  51-60            4745.0  31-40
3        4             26                  b  ...  51-60            7272.0  41-50
4        5             26                  b  ...  51-60            5558.0  41-50

[5 rows x 18 columns]
318438


In [ ]:
dftest = pd.read_csv('test.csv')
print(dftest.head())
print(len(dftest))

   case_id  Hospital_code  ...    Age  Admission_Deposit
0   318439             21  ...  71-80             3095.0
1   318440             29  ...  71-80             4018.0
2   318441             26  ...  71-80             4492.0
3   318442              6  ...  71-80             4173.0
4   318443             28  ...  71-80             4161.0

[5 rows x 17 columns]
137057


In [ ]:
df = pd.concat([dftrain, dftest])

In [ ]:
df.head()

,case_id,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,patientid,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,Stay
0,1,8,c,3,Z,3,radiotherapy,R,F,2.0,31397,7.0,Emergency,Extreme,2,51-60,4911.0,0-10
1,2,2,c,5,Z,2,radiotherapy,S,F,2.0,31397,7.0,Trauma,Extreme,2,51-60,5954.0,41-50
2,3,10,e,1,X,2,anesthesia,S,E,2.0,31397,7.0,Trauma,Extreme,2,51-60,4745.0,31-40
3,4,26,b,2,Y,2,radiotherapy,R,D,2.0,31397,7.0,Trauma,Extreme,2,51-60,7272.0,41-50
4,5,26,b,2,Y,2,radiotherapy,S,D,2.0,31397,7.0,Trauma,Extreme,2,51-60,5558.0,41-50


In [ ]:
df.columns
for column in df.columns:
  print(column)
  print(df[column].value_counts())

case_id
2047      1
429179    1
402542    1
404591    1
447600    1
         ..
234279    1
256808    1
254761    1
260906    1
2049      1
Name: case_id, Length: 455495, dtype: int64
Hospital_code
26    47523
23    38220
19    30036
6     29221
11    24827
14    24715
28    24572
27    20243
9     16360
12    16170
29    16158
32    15252
25    14046
10    13538
15    13326
21    11784
24    11467
3     10277
17     7844
1      7460
5      7448
13     7432
2      7277
30     7215
22     6006
31     5740
8      5284
16     5244
18     5228
20     1962
7      1871
4      1749
Name: Hospital_code, dtype: int64
Hospital_type_code
a    204730
b     98884
c     66147
e     35428
d     29048
f     15252
g      6006
Name: Hospital_type_code, dtype: int64
City_Code_Hospital
1     79058
2     74312
6     67441
7     50279
3     45544
5     44395
9     37428
11    24572
4     19778
10     7460
13     5228
Name: City_Code_Hospital, dtype: int64
Hospital_region_code
X    190849
Y    174707
Z     8

In [ ]:
la = df[df.Hospital_type_code == 'g']

In [ ]:
la.Hospital_code.value_counts()

22    6006
Name: Hospital_code, dtype: int64

In [ ]:
la.head()

,case_id,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,patientid,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,Stay
10,11,22,g,9,Y,2,radiotherapy,S,B,2.0,31397,7.0,Urgent,Extreme,2,51-60,5797.0,21-30
199,200,22,g,9,Y,4,gynecology,R,B,2.0,93250,25.0,Urgent,Moderate,2,31-40,4557.0,11-20
343,344,22,g,9,Y,2,gynecology,S,B,3.0,78271,8.0,Trauma,Extreme,2,61-70,5531.0,41-50
364,365,22,g,9,Y,4,gynecology,R,B,2.0,109166,5.0,Emergency,Moderate,2,31-40,3895.0,0-10
399,400,22,g,9,Y,2,gynecology,S,B,2.0,114256,8.0,Urgent,Extreme,3,71-80,4834.0,71-80


In [ ]:
df.isnull().any()

case_id                              False
Hospital_code                        False
Hospital_type_code                   False
City_Code_Hospital                   False
Hospital_region_code                 False
Available Extra Rooms in Hospital    False
Department                           False
Ward_Type                            False
Ward_Facility_Code                   False
Bed Grade                             True
patientid                            False
City_Code_Patient                     True
Type of Admission                    False
Severity of Illness                  False
Visitors with Patient                False
Age                                  False
Admission_Deposit                    False
Stay                                  True
dtype: bool

In [ ]:
len(df[df['Bed Grade'].isnull()])

148

In [ ]:
len(df[df['City_Code_Patient'].isnull()])

6689

In [ ]:
len(df[df['Stay'].isnull()])

137057